In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [10]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     50   10    9  111.46  490  110.40  1.05  0.01  103.82  459  102.45   
10000    47   20   19  111.00  469  109.64  1.36  0.00  103.50  440  101.89   
15000    45   30   29  110.78  459  109.27  1.50  0.00  103.56  440  101.91   
20000    59   40   39  110.64  452  109.03  1.60  0.01  103.54  435  101.77   
25000    50   50   49  110.54  447  108.87  1.67  0.01  103.55  434  101.76   
30000    55   60   59  110.46  444  108.73  1.73  0.01  103.55  430  101.67   
35000    52   70   69  110.40  441  108.62  1.77  0.00  103.55  431  101.69   
40000    60   80   79  110.35  438  108.53  1.81  0.01  103.46  426  101.51   
45000    54   90   89  110.31  436  108.46  1.85  0.01  103.46  425  101.52   
50000    59  100   99  110.27  434  108.39  1.88  0.01  103.35  421  101.34   
55000    50  110  109  110.25  433  108.34  1.90  0.01  103.21  421  101.29   
60000    55  120  119  110.22  432  108.30  1.92  0.01  103.17  416  101.16   
65000    55  130  129  110.20  431  108.26  1.94  0.01  103.10  415  101.06   
70000    52  140  139  110.18  430  108.22  1.95  0.01  103.14  415  101.11   
75000    56  150  149  110.16  429  108.18  1.97  0.00  103.07  413  101.00   
80000    53  160  159  110.14  428  108.15  1.99  0.00  103.11  414  101.05   
85000    55  170  169  110.12  427  108.11  2.00  0.00  103.05  412  100.96   
90000    55  180  179  110.10  426  108.08  2.02  0.00  103.07  411  100.96   
95000    51  190  189  110.09  426  108.06  2.03  0.00  103.15  413  101.03   
100000   54  200  199  110.08  425  108.03  2.04  0.00  103.11  412  100.98   
105000   56  210  209  110.06  424  108.01  2.06  0.00  103.10  412  100.97   
110000   52  220  219  110.05  424  107.98  2.07  0.00  103.11  412  100.99   
115000   54  230  229  110.04  423  107.96  2.08  0.00  103.14  412  101.00   
120000   55  240  239  110.03  423  107.94  2.09  0.00  103.11  412  100.98   
125000   52  250  249  110.02  422  107.93  2.10  0.00  103.11  411  100.97   
130000   55  260  259  110.01  422  107.91  2.10  0.00  102.99  408  100.83   
135000   51  270  269  110.00  421  107.89  2.11  0.00  103.01  409  100.86   
140000   56  280  279  109.99  421  107.87  2.12  0.00  102.96  406  100.78   
145000   54  290  289  109.98  420  107.85  2.13  0.00  102.98  407  100.81   
150000   55  300  299  109.97  420  107.84  2.13  0.00  102.99  408  100.83   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
350000   51  701  200  109.81  411  107.52  2.28  0.00  102.92  404  100.67   
355000   51  711  210  109.81  411  107.51  2.28  0.00  102.93  404  100.69   
360000   52  721  220  109.81  411  107.51  2.28  0.00  102.88  403  100.65   
365000   51  731  230  109.80  411  107.50  2.28  0.00  102.91  403  100.67   
370000   52  741  240  109.80  411  107.50  2.28  0.00  102.92  404  100.69   
375000   52  751  250  109.80  411  107.49  2.28  0.00  102.88  403  100.65   
380000   52  761  260  109.80  411  107.49  2.29  0.00  102.87  402  100.61   
385000   53  771  270  109.79  411  107.49  2.29  0.00  102.86  402  100.61   
390000   52  781  280  109.79  411  107.48  2.29  0.00  102.82  402  100.58   
395000   51  791  290  109.79  410  107.48  2.29  0.00  102.87  403  100.62   
400000   52  801  300  109.79  410  107.47  2.29  0.00  102.87  403  100.63   
405000   52  811  310  109.79  410  107.47  2.29  0.00  102.88  403  100.64   
410000   51  821  320  109.78  410  107.47  2.30  0.00  102.92  404  100.67   
415000   54  831  330  109.78  410  107.46  2.30  0.00  102.94  404  100.69   
420000   53  841  340  109.78  410  107.46  2.30  0.00  102.94  405  100.70   
425000   55  851  350  109.78  410  107.46  2.30  0.00  102.94  405  100.69   
430000   54  861  360  109.78  410  107.45  2.30  0.00  102.95  406  100.72   
435000   53  871  370  109.78  410  107.45  2.30  0.00  102.91  404  100.

0 R: 1.000 P: 0.402 carry pockets room pocket small nice perfect strap hold space
   1 R: 0.227 P: 0.094 sleeve inch - : zipper quality price bought nice bit
     12 R: 0.044 P: 0.044 & ; ! chromebook perfectly sleeve love pro air inside
     11 R: 0.089 P: 0.089 sleeve protection air pro protect inside soft scratches nice perfectly
   2 R: 0.178 P: 0.062 months broke zipper weeks time quality handle year back started
     21 R: 0.049 P: 0.049 cover keyboard easy mac hard screen love apple protector nice
     23 R: 0.067 P: 0.067 bottom top plastic part piece corners back cover rubber speck
   3 R: 0.076 P: 0.035 bought quality school price gift college years year purchased amazon
     31 R: 0.041 P: 0.041 ! love perfect recommend ... color buy absolutely awesome compliments
   4 R: 0.117 P: 0.044 item return amazon smell received $ ! shipping ordered arrived
     41 R: 0.073 P: 0.073 color cover pro perfectly picture ! mac love apple blue
